In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

db = database('trfc.db')

app, rt = fast_app()

# /season/record

In [2]:
def get_season_record(season):

    params = [season]

    query = f"""
        SELECT 
            r.season,
            r.competition,
            COUNT(*) as P,
            COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) as W,
            COUNT(CASE WHEN r.outcome = 'D' THEN 1 END) as D,
            COUNT(CASE WHEN r.outcome = 'L' THEN 1 END) as L,
            SUM(r.goals_for) as GF,
            SUM(r.goals_against) as GA,
            SUM(r.goals_for) - SUM(r.goals_against) as GD,
            ROUND(CAST(COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) AS FLOAT) / COUNT(*) * 100, 2) as win_pc
        FROM results r
        LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
            AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
        LEFT JOIN managers m ON mr.manager_id = m.manager_id
        LEFT JOIN season_league_tiers slt ON r.season = slt.season AND r.game_type = 'League'
        WHERE r.season = ?
        GROUP BY r.season, r.competition
        ORDER BY r.season DESC, P DESC
    """

    cursor = db.conn.execute(query, params)
    columns = [description[0] for description in cursor.description]
    return [dict(zip(columns, record)) for record in cursor.fetchall()]

ssn = get_season_record('2024/25')

pd.DataFrame(ssn)

,season,competition,P,W,D,L,GF,GA,GD,win_pc
0,2024/25,League Two,3,1,2,0,1,0,1,33.33
1,2024/25,Carabao Cup,2,1,0,1,3,4,-1,50.00
2,2024/25,Bristol Street Motors Trophy,1,0,0,1,1,3,-2,0.00


# /season/results

In [105]:
def get_season_results(season):

    params = [season]

    query = f"""
        SELECT *,
            COALESCE(m.manager_name, 'Unknown') as manager_name
        FROM results r        
        LEFT JOIN season_league_tiers slt 
            ON r.season = slt.season 
        LEFT JOIN manager_reigns mr 
            ON r.game_date >= mr.mgr_date_from 
            AND r.game_date <= mr.mgr_date_to
        LEFT JOIN managers m 
            ON mr.manager_id = m.manager_id
        LEFT JOIN cup_game_details cgd
            ON r.game_date = cgd.cup_game_date
        WHERE r.season = ?
        ORDER BY r.game_date
    """

    cursor = db.conn.execute(query, params)
    columns = [description[0] for description in cursor.description]
    return [dict(zip(columns, record)) for record in cursor.fetchall()]

res = get_season_results('1980/81')

pd.DataFrame(res).head(3)

,game_date,season,game_no,opposition,venue,score,outcome,goals_for,goals_against,goal_diff,...,game_length,outcome_desc,is_qualifying_round,is_group_stage,is_multi_leg,is_replay,is_extra_time,is_golden_goal,is_pen_shootout,is_play_off
0,1980-08-09,1980/81,1,Port Vale,A,3-2,W,3,2,1,...,90.0,None,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1980-08-12,1980/81,2,Port Vale,H,0-1,L,0,1,-1,...,120.0,Won on away goals rule,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1980-08-16,1980/81,3,Hereford United,A,1-1,D,1,1,0,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
# def add_tier(row):
#     if row['generic_comp'] in ['Football League', 'Non-League']:
#         if 'play-off' in row['competition'].lower():
#             return row['generic_comp'] + ' ' + str(int(row['league_tier'])) + ' [P/O]'
#         else:
#             return row['generic_comp'] + ' ' + str(int(row['league_tier']))
#     else:
#         return row['generic_comp']

# league_tiers = pd.read_csv('./data/season_league_tiers.csv')
# results = pd.read_csv('./data/results.csv')

# results = results.merge(league_tiers, on='season', how='left').copy()

# results['generic_comp'] = results.apply(add_tier, axis=1)

# results.to_csv('./data/results.csv', index=False)

In [90]:
# po_dates = results[results['generic_comp'].str.contains('P/O')].game_date

# cgd = pd.read_csv('./data/cup_game_details.csv')

# cgd.loc[cgd.cup_game_date.isin(po_dates), 'is_play_off'] = 1
# cgd.loc[~cgd.cup_game_date.isin(po_dates), 'is_play_off'] = 0

# cgd['is_play_off'] = cgd['is_play_off'].astype(int)

# cgd.to_csv('./data/cup_game_details.csv', index=False)

In [16]:
def get_goals(seasons):
    # Convert single season to list if string is passed
    if isinstance(seasons, str):
        seasons = [seasons]
    
    # Create the placeholders using standard SQLite parameter style
    placeholders = ','.join('?' * len(seasons))
    
    query = f"""
        SELECT 
            r.season,
            COUNT(DISTINCT g.player_id) as unique_scorers
        FROM results r
        INNER JOIN goals g on r.game_date = g.game_date
        WHERE r.season IN ({placeholders})
        GROUP BY r.season
    """
    
    # Pass the seasons list directly as parameters
    cursor = db.conn.execute(query, seasons)
    columns = [description[0] for description in cursor.description]
    return [dict(zip(columns, record)) for record in cursor.fetchall()]

# Example usage:
# Single season
gls_single = get_goals('2024/25')

# Multiple seasons
gls_multiple = get_goals(['2024/25', '2023/24', '2022/23'])

# Convert to DataFrame
df = pd.DataFrame(gls_multiple)

df

,season,unique_scorers
0,2022/23,15
1,2023/24,12
2,2024/25,4
